In [1]:
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import pybaseball
from pybaseball.cache import cache
import datetime
import warnings

warnings.simplefilter("ignore")
cache.enable()

ModuleNotFoundError: No module named 'pybaseball'

In [ ]:
# Load the original training and test data
train = pd.read_csv('cleaned_data/train_clean.csv')
test = pd.read_csv('cleaned_data/test_clean.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train['outcome'].head()

In [ ]:
train.drop('outcome',axis=1,inplace=True)

In [ ]:
pd.concat([train,test],axis=1).duplicated().any().any()

In [ ]:
cat_features = test.select_dtypes(include='O').columns.tolist()
initial_features = list(test.columns)
for feature in initial_features:
    if feature in cat_features:    
        categories = sorted(set(test[feature].dropna()))
        dtype = pd.CategoricalDtype(categories=categories, ordered=False)
        print(sum(~np.isin(test[feature].unique(),train[feature].unique())))
        print(test[feature].unique()[~np.isin(test[feature].unique(),train[feature].unique())])
        train.loc[~train[feature].isin(categories),feature] = np.nan
        train[feature] = train[feature].astype(dtype)     
        test[feature] = test[feature].astype(dtype)

In [ ]:
drop_cols = ['game_year','game_type']
target = 'outcome_code'

In [ ]:
train.drop(drop_cols,axis=1,inplace=True)
test.drop(drop_cols,axis=1,inplace=True)
cat_features = test.select_dtypes(include='category').columns.tolist()
initial_features = list(test.columns)

In [ ]:
_, axs = plt.subplots(1,2, figsize=(8,3))
ax = axs.ravel()
vc = train[target].value_counts()/len(train)
ax[0].bar(vc.index,vc,color='lightblue')
ax[0].bar_label(ax[0].containers[0], label_type='center', color='red')
ax[0].yaxis.set_major_formatter('{x:.0%}')
vc = train[target].value_counts()
ax[1].pie(vc,labels=vc.index,autopct='%.0f%%')
plt.title('Targets');

In [ ]:
import uuid
# Function to get additional data using pybaseball
def get_additional_data(existing_uids):
    HIT_COLS = ["home_run", "triple", "double", "single"]
    
    def spray_angle(hc_x, hc_y):
        return np.arctan2(hc_x - 125.42, 198.27 - hc_y)
    
    additional_data = pybaseball.statcast(start_dt="2024-07-17", end_dt=datetime.datetime.now().date().strftime("%Y-%m-%d"))
    
    additional_data = additional_data[additional_data['type'] == 'X']
    additional_data['is_lhp'] = additional_data['p_throws'] == 'L'
    additional_data['is_lhb'] = additional_data['stand'] == 'L'
    additional_data['outcome'] = additional_data['events'].map(
        lambda x: 'out' if x not in HIT_COLS else x)
    additional_data['outcome_code'] = additional_data['outcome'].map(
        {'out': 0, 'single': 1, 'double': 2, 'triple': 3, 'home_run': 4})
    additional_data['is_top'] = additional_data['inning_topbot'] == 'TOP'
    additional_data['spray_angle'] = np.where(
        additional_data['is_lhb'],
        -spray_angle(additional_data['hc_x'], additional_data['hc_y']),
        spray_angle(additional_data['hc_x'], additional_data['hc_y'])
    )
    
    additional_data = additional_data[additional_data['outcome'].isin(HIT_COLS)]
    
    # Generate new unique UIDs
    new_uids = [str(uuid.uuid4()) for _ in range(len(additional_data))]
    additional_data['uid'] = new_uids
    
    # Select only the columns present in the original training data
    selected_columns = [col for col in train.columns if col in additional_data.columns]
    
    return additional_data[selected_columns]

# Get the existing UIDs from the original training data
existing_uids = set(train['uid'])

# Combine original and additional data
additional_data = get_additional_data(existing_uids)
all_train = pd.concat([train, additional_data], ignore_index=True)

# Verify that we have the correct number of unique UIDs
print(f"Total rows in combined dataset: {len(all_train)}")
print(f"Number of unique UIDs: {all_train['uid'].nunique()}")

# Continue with your existing preprocessing steps
cat_features = test.select_dtypes(include='object').columns.tolist()
initial_features = list(test.columns)
for feature in initial_features:
    if feature in cat_features:    
        categories = sorted(set(test[feature].dropna()))
        dtype = pd.CategoricalDtype(categories=categories, ordered=False)
        print(sum(~np.isin(test[feature].unique(), all_train[feature].unique())))
        print(test[feature].unique()[~np.isin(test[feature].unique(), all_train[feature].unique())])
        all_train.loc[~all_train[feature].isin(categories), feature] = np.nan
        all_train[feature] = all_train[feature].astype(dtype)     
        test[feature] = test[feature].astype(dtype)

target = 'outcome_code'

cat_features = test.select_dtypes(include='category').columns.tolist()
initial_features = list(test.columns)

# Visualization
_, axs = plt.subplots(1, 2, figsize=(8, 3))
ax = axs.ravel()
vc = all_train[target].value_counts(normalize=True)
ax[0].bar(vc.index, vc, color='lightblue')
ax[0].bar_label(ax[0].containers[0], label_type='center', color='red')
ax[0].yaxis.set_major_formatter('{x:.0%}')
vc = all_train[target].value_counts()
ax[1].pie(vc, labels=vc.index, autopct='%.0f%%')
plt.title('Targets')
plt.show()

In [ ]:
def add_features(df):
    df = df.copy()
    
    # First set of features
    df['pitch_speed_diff'] = df['release_speed'] - df['effective_speed']
    df['horizontal_movement'] = df['pfx_x'] * df['release_extension']
    df['vertical_movement'] = df['pfx_z'] * df['release_extension']
    df['pitch_location'] = np.sqrt(df['plate_x']**2 + df['plate_z']**2)
    df['count_pressure'] = df['balls'] - df['strikes'] + 1
    df['bat_pitch_speed_ratio'] = df['bat_speed'] / df['release_speed']
    df['swing_efficiency'] = df['bat_speed'] / df['swing_length']
    df['movement_complexity'] = np.sqrt(df['pfx_x']**2 + df['pfx_z']**2)
    df['release_angle'] = np.arctan2(df['release_pos_z'], df['release_pos_x'])
    df['approach_angle'] = np.arctan2(df['vz0'], np.sqrt(df['vx0']**2 + df['vy0']**2))
    df['spin_efficiency'] = np.abs(np.sin(np.radians(df['spin_axis'])))
    df['magnus_effect'] = df['release_spin_rate'] * df['movement_complexity']
    df['break_time'] = df['release_extension'] / df['effective_speed']
    df['tunneling_time'] = 0.15 * df['release_speed'] / (df['movement_complexity'] + 1e-5)  # Add small constant to avoid division by zero
    df['stance_advantage'] = ((df['is_lhp'] == 1) & (df['is_lhb'] == 0)) | ((df['is_lhp'] == 0) & (df['is_lhb'] == 1))
    df['pitch_speed_change'] = df.groupby('pitch_number')['release_speed'].diff().fillna(0)
    df['horizontal_change'] = df.groupby('pitch_number')['pfx_x'].diff().fillna(0)
    df['vertical_change'] = df.groupby('pitch_number')['pfx_z'].diff().fillna(0)
    df['baserunner_pressure'] = df['on_1b'].fillna(0) + 2*df['on_2b'].fillna(0) + 3*df['on_3b'].fillna(0)
    df['late_inning_pressure'] = (df['inning'] >= 7).astype(int)
    df['release_x_deviation'] = df['release_pos_x'] - df['release_pos_x'].mean()
    df['release_z_deviation'] = df['release_pos_z'] - df['release_pos_z'].mean()
    df['relative_height'] = (df['plate_z'] - df['sz_bot']) / (df['sz_top'] - df['sz_bot'])
    
    # Handle NA values for is_strike
    df['is_strike'] = (
        (df['plate_x'].between(-0.7083, 0.7083)) & 
        (df['relative_height'].between(0, 1))
    ).fillna(False).astype(int)
    
    df['contact_quality'] = df['bat_speed'] * np.cos(np.radians(df['spray_angle']))

    # New hitting-focused features
    df['bat_to_pitch_speed_ratio'] = df['bat_speed'] / df['effective_speed']
    df['swing_plane_efficiency'] = np.cos(np.radians(df['spray_angle'])) / (df['swing_length'] + 1e-5)
    df['contact_zone'] = np.arctan2(df['plate_z'], df['plate_x']) + df['spray_angle']
    df['swing_decision_time'] = df['effective_speed'] / (df['swing_length'] + 1e-5)
    df['power_potential'] = df['bat_speed'] * np.abs(np.sin(np.radians(df['spray_angle'])))
    df['plate_discipline_index'] = df['is_strike'] * df['swing_efficiency']
    df['bat_control'] = df['bat_speed'] / (df['swing_length'] + 1e-5)
    df['exit_velocity_estimate'] = (df['bat_speed'] + df['effective_speed']) * 0.5
    df['launch_angle_estimate'] = df['spray_angle'] * 0.6
    df['sweet_spot_probability'] = 1 / (1 + np.exp(-(df['bat_speed'] - 70) / 10)) * (1 - np.abs(df['swing_length'] - 24) / 24)

    # Additional features
    df['exit_velocity_launch_angle_ratio'] = df['exit_velocity_estimate'] / (df['launch_angle_estimate'].abs() + 1)
    df['swing_efficiency_score'] = df['swing_efficiency'] * df['bat_control'] * df['sweet_spot_probability']
    df['pitch_complexity'] = df['release_speed'] * df['spin_efficiency'] * (df['vertical_movement'].abs() + 1)
    df['batter_readiness'] = df['plate_discipline_index'] * df['swing_decision_time']
    df['contact_quality_score'] = df['exit_velocity_estimate'] * df['sweet_spot_probability'] * df['bat_speed']
    df['pitch_location_effectiveness'] = df['pitch_location'] * df['is_strike'] * (1 - df['plate_discipline_index'])
    df['count_pressure_impact'] = df['count_pressure'] * (df['balls'] - df['strikes'] + 3)
    df['batter_power_index'] = df['power_potential'] * df['exit_velocity_estimate'] * df['bat_speed']
    df['pitch_deception_score'] = df['effective_speed'] / df['release_speed'] * df['spin_efficiency']
    df['batter_contact_ability'] = df['bat_control'] * df['contact_zone'] * df['swing_plane_efficiency']
    df['pitch_movement_complexity'] = (df['pfx_z'].abs() + df['vertical_change'].abs()) * df['spin_efficiency']
    df['batter_pitcher_matchup'] = df['stance_advantage'] * df['relative_height'] * df['pitch_location_effectiveness']
    df['swing_aggressiveness'] = df['swing_length'] * df['bat_speed'] / (df['swing_decision_time'] + 1e-5)
    df['pitch_bat_speed_differential'] = df['release_speed'] - df['bat_speed']
    df['exit_velocity_pitch_speed_ratio'] = df['exit_velocity_estimate'] / df['release_speed']
    df['vertical_approach_effectiveness'] = df['approach_angle'] * df['launch_angle_estimate'] * df['exit_velocity_estimate']
    df['horizontal_spray_effectiveness'] = df['spray_angle'].abs() * df['exit_velocity_estimate']
    df['strike_zone_coverage'] = (df['sz_top'] - df['sz_bot']) * df['plate_discipline_index']
    df['pitch_location_accuracy'] = 1 - (((df['plate_x']**2 + df['plate_z']**2)**0.5) / ((2.5**2 + 2.5**2)**0.5))

    return df


In [ ]:
# Apply the feature addition to the dataframes
all_train = add_features(all_train)
test = add_features(test)

# Display some information about the new features
print(f"Number of features after engineering: {all_train.shape[1]}")
print("\nNew features added:")
new_features = set(all_train.columns) - set(initial_features)
print(", ".join(new_features))

In [ ]:
X = all_train.drop(['outcome_code', 'uid'], axis=1)  # Assuming 'uid' is not a feature
y = all_train['outcome_code']

In [ ]:
# Identify categorical features
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Handle NaN values in categorical features
for feature in cat_features:
    # For training data
    if X[feature].dtype.name == 'category':
        # Add 'Unknown' to categories if it's not already there
        if 'Unknown' not in X[feature].cat.categories:
            X[feature] = X[feature].cat.add_categories('Unknown')
        X[feature] = X[feature].fillna('Unknown')
    else:
        X[feature] = X[feature].fillna('Unknown').astype('category')
    
    # For test data
    if test[feature].dtype.name == 'category':
        # Ensure all categories from training data are in test data
        test[feature] = test[feature].cat.add_categories(X[feature].cat.categories.difference(test[feature].cat.categories))
        if 'Unknown' not in test[feature].cat.categories:
            test[feature] = test[feature].cat.add_categories('Unknown')
        test[feature] = test[feature].fillna('Unknown')
    else:
        test[feature] = test[feature].fillna('Unknown').astype('category')
    
    # Ensure both train and test have the same categories
    all_categories = X[feature].cat.categories.union(test[feature].cat.categories)
    X[feature] = X[feature].cat.set_categories(all_categories)
    test[feature] = test[feature].cat.set_categories(all_categories)

# Handle NaN values in numerical features
# Separate integer and float features
int_features = X.select_dtypes(include=['int64']).columns.tolist()
float_features = X.select_dtypes(include=['float64']).columns.tolist()

# Convert integer features to float to accommodate mean values
X[int_features] = X[int_features].astype('float64')
test[int_features] = test[int_features].astype('float64')

# Combine all numerical features as float
num_features = int_features + float_features

# Fill NaN values with the mean from the training set
X[num_features] = X[num_features].fillna(X[num_features].mean())
test[num_features] = test[num_features].fillna(X[num_features].mean())  # Use training mean for test set


In [ ]:
# Double-check for any remaining NaN values
assert X.isnull().sum().sum() == 0, "There are still NaN values in the training set"
assert test.isnull().sum().sum() == 0, "There are still NaN values in the test set"

# Print info about the datasets
print(X.info())
print("\n")
print(test.info())

In [ ]:
import optuna
from optuna.samplers import TPESampler
import catboost as cb
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
print("Categorical Features:", cat_features)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
def custom_cross_validate(model, X, y, cv, cat_features):
    """
    Performs cross-validation for CatBoostClassifier with categorical features.

    Parameters:
    - model: The CatBoostClassifier instance.
    - X: Feature matrix (pandas DataFrame).
    - y: Target vector (pandas Series).
    - cv: Cross-validation splitter (e.g., StratifiedKFold instance).
    - cat_features: List of categorical feature indices or names.

    Returns:
    - oof_preds: Out-of-fold predictions as a NumPy array.
    """
    oof_preds = np.zeros((len(X), len(y.unique())))
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Fit the model with categorical features
        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=50,
            verbose=False,
            cat_features=cat_features
        )
        
        # Predict probabilities for the validation set
        oof_preds[val_idx] = model.predict_proba(X_val)
        
        # Calculate and print AUC for the current fold
        score = roc_auc_score(y_val, oof_preds[val_idx], multi_class='ovr')
        print(f"Fold {fold + 1} - AUC: {score:.4f}")
    
    return oof_preds


In [ ]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for CatBoost
    cb_params = {
        'iterations': trial.suggest_int('cb_iterations', 500, 2000),
        'learning_rate': trial.suggest_loguniform('cb_learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('cb_depth', 4, 10),
        'l2_leaf_reg': trial.suggest_int('cb_l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_uniform('cb_bagging_temperature', 0, 1),
        'random_state': 42,
        'eval_metric': 'AUC',
        'verbose': False,
    }
    
    # Initialize CatBoost with suggested hyperparameters
    cb_model = cb.CatBoostClassifier(**cb_params)
    
    # Perform cross-validation for CatBoost
    cb_oof = custom_cross_validate(cb_model, X, y, kf, cat_features)
    cb_score = roc_auc_score(y, cb_oof, multi_class='ovr')
    print(f"CatBoost OOF AUC: {cb_score:.4f}")
    
    # Suggest hyperparameters for XGBoost
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 500),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
        'subsample': trial.suggest_uniform('xgb_subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('xgb_colsample_bytree', 0.5, 1.0),
        'objective': 'multi:softprob',
        'num_class': len(y.unique()),
        'random_state': 42,
        'verbosity': 0
    }
    
    # Initialize XGBoost with suggested hyperparameters
    xgb_model = xgb.XGBClassifier(**xgb_params)

    # Use cross_val_predict for out-of-fold predictions
    ensemble_oof = cross_val_predict(xgb_model, cb_oof, y, cv=kf, method='predict_proba')
    ensemble_score = roc_auc_score(y, ensemble_oof, multi_class='ovr')
    print(f"XGBoost Ensemble OOF AUC: {ensemble_score:.4f}")
    
    return ensemble_score

In [ ]:
# Create an Optuna study
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
# Optimize the study
study.optimize(objective, n_trials=10, timeout=3600)  # Adjust n_trials and timeout as needed

# Get the best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

In [ ]:
# ---------------------------
# 5. Training Final Models with Best Hyperparameters
# ---------------------------

# Define the best CatBoost model
best_cb_params = {
    'iterations': best_params.get('cb_iterations', 1418),
    'learning_rate': best_params.get('cb_learning_rate', .01607712851203988),
    'depth': best_params.get('cb_depth', 6),
    'l2_leaf_reg': best_params.get('cb_l2_leaf_reg', 4),
    'bagging_temperature': best_params.get('cb_bagging_temperature', 0.45606998421703593),
    'random_state': 42,
    'eval_metric': 'AUC',
    'verbose': False,
    # 'cat_features' is removed from here
}

cb_model = cb.CatBoostClassifier(**best_cb_params)

# Train the best CatBoost model using the custom cross-validation
print("Training optimized CatBoost model...")
cb_oof = custom_cross_validate(cb_model, X, y, kf, cat_features)
cb_score = roc_auc_score(y, cb_oof, multi_class='ovr')
print(f"Optimized CatBoost OOF AUC: {cb_score:.4f}")

# Define the best XGBoost model
best_xgb_params = {
    'n_estimators': best_params.get('xgb_n_estimators', 414),
    'learning_rate': best_params.get('xgb_learning_rate', 0.019721610970574007),
    'max_depth': best_params.get('xgb_max_depth', 7),
    'subsample': best_params.get('xgb_subsample', .7962072844310213),
    'colsample_bytree': best_params.get('xgb_colsample_bytree', .5232252063599989),
    'objective': 'multi:softprob',
    'num_class': len(y.unique()),
    'random_state': 42,
    'verbosity': 0
}

xgb_model = xgb.XGBClassifier(**best_xgb_params)

# Train the XGBoost ensemble model using cross_val_predict
print("Training optimized XGBoost Ensemble...")
ensemble_oof = cross_val_predict(
    xgb_model, 
    cb_oof, 
    y, 
    cv=kf, 
    method='predict_proba'
)
ensemble_score = roc_auc_score(y, ensemble_oof, multi_class='ovr')
print(f"Optimized XGBoost Ensemble OOF AUC: {ensemble_score:.4f}")

In [ ]:
# Final predictions on test set
print("Making final predictions on the test set...")
cb_model.fit(X, y, cat_features=cat_features, verbose=False)
cb_test_preds = cb_model.predict_proba(test.drop('uid', axis=1))

# Train XGBoost on the full OOF predictions
xgb_model.fit(cb_oof, y)
final_preds = xgb_model.predict_proba(cb_test_preds)

In [ ]:
# Prepare submission
submission = pd.DataFrame({
    'uid': test['uid'],
    'out': final_preds[:, 0],
    'single': final_preds[:, 1],
    'double': final_preds[:, 2],
    'triple': final_preds[:, 3],
    'home_run': final_preds[:, 4]
})

In [ ]:
submission.to_csv('submission.csv', index=False)